In [1125]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [1126]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
665,tt0852713,25000000,70442940,The House Bunny,Anna Faris|Emma Stone|Kat Dennings|Katharine M...,Fred Wolf,For the girls of ZETA house college life was n...,Shelley is living a carefree life until a riva...,97,Romance|Comedy,Columbia Pictures|Happy Madison Productions|Re...,8/22/2008,5.6,2008
666,tt1132626,10800000,113857533,Saw V,Tobin Bell|Costas Mandylor|Scott Patterson|Bet...,David Hackl,You Won't Believe How It Ends,Detective Hoffman is seemingly the last person...,92,Horror|Thriller|Crime,Lionsgate|Twisted Pictures,10/23/2008,5.9,2008
908,tt0277296,60000000,165333180,The Scorpion King,Dwayne Johnson|Kelly Hu|Michael Clarke Duncan|...,Chuck Russell,Warrior. Legend. King.,"In ancient Egypt, peasant Mathayus is hired to...",92,Action|Fantasy|Adventure,Universal Pictures|Kalima Productions GmbH & C...,4/16/2002,5.2,2002
1382,tt0369441,100000000,202026112,Fun with Dick and Jane,Jim Carrey|TÃ©a Leoni|Alec Baldwin|Richard Jen...,Dean Parisot,Giving big businesses a run for their money!,"After losing their high-paying corporate jobs,...",90,Comedy,Imagine Entertainment|Columbia Pictures Corpor...,12/21/2005,5.9,2005
245,tt0438488,200000000,371353001,Terminator Salvation,Christian Bale|Sam Worthington|Anton Yelchin|M...,McG,The End Begins.,"All grown up in post-apocalyptic 2018, John Co...",115,Drama|Action|Thriller|Science Fiction,Columbia Pictures|The Halcyon Company|Wonderla...,5/20/2009,5.8,2009


In [1127]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1128]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...


def year_detect(release_date):
    result = re.search(r'\d{4}', release_date)
    if result:
        return int(result.group(0))
    else:
        return 1970


def month_detect(release_date):
    result = re.search(r'^\d\d?', release_date)
    if result:
        month = int(result.group(0))
        if month < 1 or month > 12:
            month = 0
        return month
    else:
        return 0


def series_aggregation_first_column (incoming_data, agg_type='sum', top=5, sort_direction=False,):
    
    result_series = pd.Series(dtype=object)

    for column1, column2 in incoming_data.items():
        
        if agg_type == 'sum':
            agg_parametr = column2
        else:
            agg_parametr = 1
            
        for item in column1.split('|'):
            
            if item in result_series.index:
                result_series[item] += agg_parametr
            else:
                result_series[item] = agg_parametr

    return result_series.sort_values(ascending=sort_direction).head(top)
  




data['profit'] = data.revenue - data.budget

#data['release_year'] = data.release_date.apply(lambda x: str(x)[-4:])

data['release_year'] = data.release_date.apply(year_detect)

data['release_month'] = data.release_date.apply(month_detect)

data['word_count_overiew'] = data.overview.apply(lambda s: len(re.findall(r'\w+', s.lower())))


In [1129]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,word_count_overiew
1471,tt0427229,50000000,88715192,Failure to Launch,Matthew McConaughey|Sarah Jessica Parker|Zooey...,Tom Dey,"To leave the nest, some men just need a little...","Tripp, an attractive man in his thirties, is s...",97,Comedy,Paramount Pictures|Scott Rudin Productions|Fai...,3/10/2006,5.6,2006,38715192,3,74
81,tt3450650,30000000,107597242,Paul Blart: Mall Cop 2,Kevin James|Raini Rodriguez|Eduardo VerÃ¡stegu...,Andy Fickman,Vegas has a new high roller.,Security guard Paul Blart is headed to Las Veg...,94,Action|Adventure|Comedy|Family,Columbia Pictures|Happy Madison Productions|So...,4/16/2015,5.3,2015,77597242,4,44
990,tt1409024,225000000,624026776,Men in Black 3,Will Smith|Tommy Lee Jones|Josh Brolin|Michael...,Barry Sonnenfeld,They are back... in time.,Agents J (Will Smith) and K (Tommy Lee Jones) ...,106,Action|Comedy|Science Fiction,Amblin Entertainment|Media Magik Entertainment...,5/23/2012,6.1,2012,399026776,5,111
1222,tt2191701,80000000,246984278,Grown Ups 2,Adam Sandler|Kevin James|Chris Rock|David Spad...,Dennis Dugan,Just because they're a little older doesn't me...,The all-star comedy cast from Grown Ups return...,100,Comedy,Columbia Pictures|Happy Madison Productions|So...,7/11/2013,5.9,2013,166984278,7,63
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014,603312399,7,28


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [1130]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# запишите свой вариант ответа


In [1131]:
# тут пишем ваш код для решения данного вопроса:

movie_name = data[data.budget == data.budget.max()].original_title.max()
movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")


Pirates of the Caribbean: On Stranger Tides (tt1298650)


In [1132]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [1133]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [1134]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать


In [1135]:

movie_name = data[data.runtime == data.runtime.max()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

Gods and Generals (tt0279111)


In [1136]:
# +
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1137]:
movie_name = data[data.runtime == data.runtime.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

Winnie the Pooh (tt1449283)


In [1138]:
# +
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1139]:
round(data.runtime.mean())

110

In [1140]:
# +
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [1141]:
round(data.runtime.median())

107

In [1142]:
# +
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1143]:
# лучше код получения столбца profit вынести в Предобработку что в начале

movie_name = data[data.profit == data.profit.max()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

Avatar (tt0499549)


In [1144]:
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1145]:
movie_name = data[data.profit == data.profit.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

The Lone Ranger (tt1210819)


In [1146]:
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1147]:
len(data[data.profit > 0])

1478

In [1148]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1149]:
movie_name = data[data.profit == data[data.release_date.str.contains('2008', na='')].profit.max()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

The Dark Knight (tt0468569)


In [1150]:
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1151]:
movie_name = data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")

The Lone Ranger (tt1210819)


In [1152]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [1153]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)'}

# 11. Какого жанра фильмов больше всего?

In [1154]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

genre_series = pd.Series(data=[0], index=['none'])

for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        if genre in genre_series.index:
            genre_series[genre] += genre_count
        else:
            genre_series[genre] = genre_count

genre_series.sort_values(ascending=False).head(5)


Drama        782
Comedy       683
Thriller     596
Action       582
Adventure    415
dtype: int64

In [1155]:
answers['11'] = 'Drama'

ВАРИАНТ 2

можно сделать чере словарь..

In [1156]:
genre_counter = Counter()

for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        genre_counter[genre] += genre_count

genre_counter
# TODO - поиск максимального значения


Counter({'Comedy': 683,
         'Drama': 782,
         'Romance': 308,
         'Horror': 176,
         'Thriller': 596,
         'Action': 582,
         'Crime': 315,
         'Family': 260,
         'Animation': 139,
         'History': 62,
         'Science Fiction': 248,
         'Adventure': 415,
         'Fantasy': 222,
         'Mystery': 168,
         'Music': 64,
         'War': 58,
         'Documentary': 8,
         'Western': 19,
         'Foreign': 2})

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1157]:

a = data[data.profit > 0].groupby(['genres'])['profit'].count()

series_aggregation_first_column(a, 'sum')


Drama        560
Comedy       551
Thriller     446
Action       444
Adventure    337
dtype: int64

In [1158]:
series_aggregation_first_column(a, 'sum').index[0]

'Drama'

In [1159]:
# answers['12'] = 'Adventure'
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [1160]:
director_revenue = pd.Series(data=[0], index=['none'])

for director_all, revenue in data.groupby(['director'])['revenue'].sum().items():
    for director in director_all.split('|'):
        if director in director_revenue.index:
            director_revenue[director] += revenue
        else:
            director_revenue[director] = revenue

director_revenue.sort_values(ascending=False).head(3)

Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
dtype: int64

In [1161]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1162]:

director_genres = data[ ['director','genres'] ][data.genres.str.contains('Action', na=False)]

director_genres = director_genres.groupby(['director'])['genres'].count()

series_aggregation_first_column(director_genres, 'sum', top=5)


Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
dtype: int64

In [1163]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1164]:

top_actor_2012 = data.query('release_year == 2012').groupby(['cast'])['revenue'].sum()

series_aggregation_first_column( top_actor_2012 ,'sum', top=1)


Chris Hemsworth    2027450773
dtype: int64

In [1165]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1166]:
# высокобюджетные - фильмы, где бюджет выше среднего по данной выборке

top_actor = data[data.budget > data.budget.mean()].groupby(['cast'])['revenue'].count()

series_aggregation_first_column( top_actor ,'sum', top=6)


Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Samuel L. Jackson    15
Eddie Murphy         15
Tom Cruise           15
dtype: int64

In [1167]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1168]:
favorite_genre = data[data.cast.str.contains('Nicolas Cage', na=False)].groupby(['genres'])['cast'].count()

series_aggregation_first_column( favorite_genre ,'sum', top=5)

Action      17
Thriller    15
Drama       12
Crime       10
Fantasy      8
dtype: int64

In [1169]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [1170]:
movie_name = data[data.profit == data[data.production_companies.str.contains('Paramount Pictures')].profit.min()].original_title.max()

movie_id = data[data.original_title == movie_name].imdb_id.max()

print(f"{movie_name} ({movie_id})")



K-19: The Widowmaker (tt0267626)


In [1171]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1172]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).head(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

In [1173]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1174]:
wb_profit = data[data.production_companies.str.contains('Warner Bros', na=False)]

wb_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

In [1175]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1176]:
# Вариант 1
data.release_month.value_counts().sort_values().tail(1)

9    227
Name: release_month, dtype: int64

In [1177]:
# Вариант 2
data.groupby(['release_month'])['imdb_id'].count().sort_values().tail(1)

release_month
9    227
Name: imdb_id, dtype: int64

In [1178]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1179]:
# data.query('6<=release_month<=8').shape
# len(data.query('6 <= release_month <= 8'))

len(data.query('release_month in [6, 7, 8]'))

450

In [1180]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1181]:

winter_director = data.query('release_month in [1, 2, 12]').director.value_counts()

series_aggregation_first_column(winter_director,'sum', top=3)

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
dtype: int64

In [1182]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1183]:

max_len = data.original_title.str.len().max()

data[data.original_title.str.len() == max_len].production_companies.iloc[0]



'Twentieth Century Fox Film Corporation|Four By Two Productions'

In [1184]:
# series_aggregation_first_column( _ ,'sum', top=6)
answers['24'] = 'Four By Two Productions' 

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1185]:
# production_companies_result = pd.DataFrame(columns=['production_companies', 'overview_word_count'])

# prod_companies = data[['production_companies', 'overview']].head(50)

# for i in prod_companies.index:
#     companies = prod_companies.iloc[i].production_companies
#     overview = prod_companies.iloc[i].overview
#     word_count = len(re.findall(r'\w+', overview.lower()))
    
#     for prod_companie in companies.split('|'):
        
#         new_location = len(production_companies_result)
        
#         production_companies_result.loc[new_location] = [prod_companie, word_count]


# production_companies_result.groupby(['production_companies'])['overview_word_count'].max()

# production_companies_result.production_companies.value_counts()
# production_companies_result.overview_word_count.max()

In [1186]:
# data['word_count_overiew']

word_count_overiew_max = data[data.word_count_overiew > data.word_count_overiew.mean()].word_count_overiew.max()

data[data.word_count_overiew == word_count_overiew_max].production_companies.iloc[0]


'Lions Gate Films|Lakeshore Entertainment|GreeneStreet Films|Midnight Picture Show'

In [1187]:
data[data.word_count_overiew == word_count_overiew_max].production_companies.iloc[0].split('|')[3]

'Midnight Picture Show'

In [1188]:
# 
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [1189]:

top_movies = ['Inside Out','The Dark Knight','12 Years a Slave']
#     'BloodRayne, The Adventures of Rocky & Bullwinkle',
#     'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down',
#     '300, Lucky Number Slevin, Kill Bill: Vol. 1',
#     'Upside Down, Inside Out, Iron Man']



In [1190]:
count_last_procent = round(len(data)*0.01)

last_procent = data.vote_average.sort_values(ascending=False).head(count_last_procent).index

data.loc[last_procent][['original_title','vote_average']]


,original_title,vote_average
599,The Dark Knight,8.1
125,The Imitation Game,8.0
9,Inside Out,8.0
118,Interstellar,8.0
34,Room,8.0
1081,The Lord of the Rings: The Return of the King,7.9
138,The Grand Budapest Hotel,7.9
1183,The Wolf of Wall Street,7.9
370,Inception,7.9
119,Guardians of the Galaxy,7.9


In [1191]:
# Проверка наличия 3-х фильмов в выборке 1% лучших фильмов
data.loc[last_procent].query('original_title in @top_movies')[['original_title','vote_average']]

top_movies = ['Upside Down','Inside Out','Iron Man']
data.query('original_title in @top_movies')[['original_title','vote_average']]

,original_title,vote_average
9,Inside Out,8.0
601,Iron Man,7.3
1007,Upside Down,6.1


In [1192]:
# 
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1193]:
a = list(combinations('carl', 3))

a


[('c', 'a', 'r'), ('c', 'a', 'l'), ('c', 'r', 'l'), ('a', 'r', 'l')]

In [1194]:
# 
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [1195]:
pair_actors = [  'Johnny Depp & Helena Bonham Carter',
                'Ben Stiller & Owen Wilson',
                'Vin Diesel & Paul Walker',
                'Adam Sandler & Kevin James',
                'Daniel Radcliffe & Rupert Grint']

actor_pair_work = pd.Series(dtype=object)

for pair_actor in pair_actors:
    actor1, actor2 =  pair_actor.split(' & ')
    pair_work_count = data[data.cast.str.contains(actor1) & data.cast.str.contains(actor2)].imdb_id.count()
    #print(pair_work_count, '|', pair_actor)
    actor_pair_work[pair_actor] = pair_work_count

actor_pair_work.sort_values(ascending=False).head(1).index[0]


'Daniel Radcliffe & Rupert Grint'

# Submission

In [1196]:
# в конце можно посмотреть свои ответы к каждому вопросу

# Вопрос № 1 ........ [ + ]
# Вопрос № 2 ........ [ + ]
# Вопрос № 3 ........ [ + ]
# Вопрос № 4 ........ [ + ]
# Вопрос № 5 ........ [ + ]
# Вопрос № 6 ........ [ + ]
# Вопрос № 7 ........ [ + ]
# Вопрос № 8 ........ [ + ]
# Вопрос № 9 ........ [ + ]
# Вопрос № 10 ....... [ + ]
# Вопрос № 11 ....... [ + ]
# Вопрос № 12 ....... [ + ]
# Вопрос № 13 ....... [ + ]
# Вопрос № 14 ....... [ + ]
# Вопрос № 15 ....... [ + ]
# Вопрос № 16 ....... [ + ]
# Вопрос № 17 ....... [ + ]
# Вопрос № 18 ....... [ + ]
# Вопрос № 19 ....... [ + ]
# Вопрос № 20 ....... [ + ]
# Вопрос № 21 ....... [ + ]
# Вопрос № 22 ....... [ + ]
# Вопрос № 23 ....... [ + ]
# Вопрос № 24 ....... [ + ]
# Вопрос № 25 ....... [ + ]
# Вопрос № 26 ....... [ + ]
# Вопрос № 27 ....... [ + ]

answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [1197]:
# и убедиться что ни чего не пропустил)
len(answers)

27